# Query the API with a user token

- We show how to obtain and use a personal token for MMODA
- We show the logging level for messages

In order to access the full functionality of MMODA, it is recommended to authenticate to the service.
This will allow the user to receive emails at job completion and to enjoy role privileges like access to
private data or elaboration of many science windows (for collaborators of UNIGE).
Moreover, in case of unexpected issues, the developers will be able to easily track the issue and
communicate with the user.

The first step is obtaining a token.
First, you need to "Sign In" on the frontend. 
[https://www.astro.unige.ch/mmoda/](https://www.astro.unige.ch/mmoda/)

### Create account

If you do not have an account create one, by clicking always on Sign-in.

<img src="Authentication_files/Login_ODA-new.png">

### login and get API code

* Once you login, you will find the "API token" button

<img src="Authentication_files/API_token.png">

* When you click on it, you will see a window in which the token can be copied on the clipboard or sent by email.

<img src="Authentication_files/token.png">

* Copy the token string and assign it to a variable called, e.g., 'token' as done in the following cell below interactively

* Note that the Token lives only 24 hours, so it needs to be regenerated if a longer time has passed since your login.


In [2]:
import getpass
token = getpass.getpass('Insert the token')

Insert the token········


## API code in email

Note that at every request that needs a sufficiently long computation, you receive an email with attached a code suitable for the python API !

## Let's get some logging

This is to help visualizing the progress.

* WANRING is the default level
* INFO writes some more information
* DEBUG is maily for developers and issue tracking

In [3]:
import logging
#default
#logging.getLogger().setLevel(logging.WARNING)
#slightly more verbose
logging.getLogger().setLevel(logging.INFO)
#all messages
#logging.getLogger().setLevel(logging.DEBUG)

logging.getLogger('oda_api').addHandler(logging.StreamHandler()) 

## Perform a query through API

You can make a query for SPI-ACS through the API and visualize the light curve as below.

Copy the API code in a cell (remove the coockie parameter if present).

Here, we have put the token value in a variable called token in a cell above.

In [5]:
from oda_api.api import DispatcherAPI

disp = DispatcherAPI(url='http://www.astro.unige.ch/mmoda/dispatch-data')
par_dict = {'src_name': '4U 1700-377', 
            'RA': '257.815417', 
            'DEC': '-41.593417', 
            'T1': '58195.455', 
            'T2': '58195.555', 
            'T_format': 'mjd', 
            'instrument': 'spi_acs', 
            'product_type': 'Real', 
            'product': 'spi_acs_lc', 
            'time_bin': '1', 
            'user_catalog_file': None,
           'token': token}

data_collection = disp.get_product(**par_dict)

/home/ferrigno/.venv/myVE/lib/python3.8/site-packages/oda_api/api.py:856: UserWarning: 
----------------------------------------------------------------------------
the parameter: T_format   is not among valid ones:
['src_name', 'RA', 'DEC', 'T1', 'T2', 'token', None, 'selected_catalog', 'time_bin']
this will throw an error in a future version 
and might break the current request!
 ----------------------------------------------------------------------------

  warnings.warn(msg)
/home/ferrigno/.venv/myVE/lib/python3.8/site-packages/oda_api/api.py:856: UserWarning: 
----------------------------------------------------------------------------
the parameter: user_catalog_file   is not among valid ones:
['src_name', 'RA', 'DEC', 'T1', 'T2', 'token', None, 'selected_catalog', 'time_bin']
this will throw an error in a future version 
and might break the current request!
 ----------------------------------------------------------------------------

  warnings.warn(msg)
- waiting for remote re

## Show data and plot

In [6]:
data_collection.show()

ID=0 prod_name=spi_acs_lc_0_query  meta_data: {'src_name': 'query', 'time_bin': 1.0, 'time': 'TIME', 'rate': 'RATE', 'rate_err': 'ERROR'}



In [7]:
lc=data_collection._p_list[0]

In [8]:
%matplotlib notebook
import matplotlib.pyplot as plt

In [9]:
plt.errorbar(lc.data_unit[1].data['TIME'], lc.data_unit[1].data['RATE'], yerr=lc.data_unit[1].data['ERROR'])
plt.xlabel('Time [IJD]')
plt.ylabel('Count rate')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'Count rate')

## Make another query

First look at the keys

In [10]:
descr = disp.get_product_description(instrument='isgri',product_name='isgri_spectrum')

--------------
parameters for product isgri_spectrum and instrument isgri

--------------
query_name: src_query
 name: src_name,  value: test,  units: str, 
 name: RA,  value: 0.0,  units: deg, 
 name: DEC,  value: 0.0,  units: deg, 
 name: T1,  value: 2001-12-11T00:00:00.000,  units: None, 
 name: T2,  value: 2001-12-11T00:00:00.000,  units: None, 
 name: token,  value: None,  units: str, 

--------------
query_name: isgri_parameters
 name: user_catalog,  value: None,  units: str, 
 name: scw_list,  value: [],  units: names_list, 
 name: selected_catalog,  value: None,  units: str, 
 name: radius,  value: 5.0,  units: deg, 
 name: max_pointings,  value: 50,  units: None, 
 name: osa_version,  value: None,  units: str, 
 name: integral_data_rights,  value: public,  units: str, 
 name: E1_keV,  value: 15.0,  units: keV, 
 name: E2_keV,  value: 40.0,  units: keV, 

--------------
query_name: isgri_spectrum_query
 product_name: isgri_spectrum


### Extract an image on a limited number of pointigs
with the parameter max_pointings, we can get a random subset of available pointings
The most common use case is an image from which we want to derive the 
catalog of bright sources in the field of view

Note that we are giving the parameters as arguments.

In [ ]:
image = disp.get_product(instrument="isgri", 
                     product="isgri_image", 
                     product_type="Real", 
                     osa_version='OSA11.0',
                     radius = 8,
                     RA=275.09142677,
                     DEC=7.18535523,
                     T1=58193.455,
                     T2=58246.892,
                     T_format= 'mjd',
                     max_pointings=10,
                     E1_keV=30.0,
                     E2_keV=80.0,
                     token=token
          )

/home/ferrigno/.venv/myVE/lib/python3.8/site-packages/oda_api/api.py:856: UserWarning: 
----------------------------------------------------------------------------
the parameter: T_format   is not among valid ones:
['src_name', 'RA', 'DEC', 'T1', 'T2', 'token', 'scw_list', 'selected_catalog', 'radius', 'max_pointings', 'osa_version', 'integral_data_rights', 'E1_keV', 'E2_keV', 'detection_threshold', 'image_scale_min', 'image_scale_max']
this will throw an error in a future version 
and might break the current request!
 ----------------------------------------------------------------------------

  warnings.warn(msg)
- waiting for remote response (since 2021-08-23 15:14:00), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96

... query status prepared => submitted
... assigned job id: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 28 seconds (22/28)

session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 301 seconds (13/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:19:03), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 310 seconds (13/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:19:12), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 320 seconds (13/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:19:22), please wai

- waiting for remote response (since 2021-08-23 15:22:19), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 \ the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 507 seconds (11/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:22:29), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 516 seconds (11/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:22:38), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 524 seconds

session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 697 seconds (9.6/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:25:38), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 705 seconds (9.6/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:25:46), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 \ the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 712 seconds (9.6/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:25:54), please 

- waiting for remote response (since 2021-08-23 15:29:04), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 911 seconds (9.3/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:29:13), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 920 seconds (9.3/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:29:22), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 930 secon

...

- waiting for remote response (since 2021-08-23 15:42:37), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 1725 seconds (8.7/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:42:47), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 \ the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 1734 seconds (8.7/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:42:55), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 1742 se

session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 1938 seconds (8.7/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:46:20), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 1948 seconds (8.7/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:46:30), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 1960 seconds (8.7/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:46:41), plea

- waiting for remote response (since 2021-08-23 15:50:23), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 \ the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2195 seconds (8.8/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:50:37), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2205 seconds (8.8/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:50:47), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2215 se

session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2433 seconds (8.9/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:54:35), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2442 seconds (8.9/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:54:44), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 \ the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2452 seconds (8.9/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:54:53), plea

- waiting for remote response (since 2021-08-23 15:58:03), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 | the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2650 seconds (8.9/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:58:12), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 / the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2659 seconds (8.9/34); in 0 SCW so far; nodes (0): 0 computed 0 restored
... 
- waiting for remote response (since 2021-08-23 15:58:21), please wait for http://www.astro.unige.ch/mmoda/dispatch-data/run_analysis
session: 4XVOH70IKSBCLCEK job: a9ab953d489bed96
 - the job is working remotely, please wait status=submitted job_id=a9ab953d in 0 messages since 2682 se

In [ ]:
#Let's look at the image
image.show()

In [ ]:
#access it as an attribute
image.mosaic_image_0_mosaic

In [ ]:
#Access it as liste memebr
data_collection._p_list[0]

In [ ]:
#Minimum detection threshold and avoid including new sources
det_sigma = 8
include_new_sources = False

sources = image.dispatcher_catalog_1.table[image.dispatcher_catalog_1.table['significance'] >= det_sigma]

if len(sources) == 0:
    print('No sources in the catalog with det_sigma > %.1f' % det_sigma)
    

if not include_new_sources:
    ind = [not 'NEW' in ss for ss in sources['src_names']]
    clean_sources = sources[ind]
else:
    clean_sources = sources

#We copy back clean sources in the image data products
image.dispatcher_catalog_1.table = clean_sources

#We derive the catalog string for the spectrum !
api_cat_str=image.dispatcher_catalog_1.get_api_dictionary()

api_cat_str

### Query a spectrum with the maximum number of available science windows

The maximum number of science windows that can be processed in a single query is 500, 
you should specify this in max_pointings. Otherwise the default value of 50 will be used.

Once the query is submitted, you will receive an email. You can now interrupt your query and wait for a second email when data will be ready.

In [ ]:
#We get a spectrum from 50 pointings, note that maximum is 500 !
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version='OSA11.0',
                     RA=275.09142677,
                         DEC=7.18535523,
                    radius = 8,
                     T1=58193.455,
                         T2=58246.892,
                         T_format= 'mjd',
                     max_pointings=50,
                     token=token,
                 selected_catalog=api_cat_str)

In [ ]:
#This is the source we inspect the spectrum for
src_name='MAXI J1820+070'

In [ ]:
#We select a particulr source
data_sel=spectrum.new_from_metadata('src_name',src_name)
data_sel.show()

In [ ]:
#We can save the files
data_sel.save_all_data()

In the folder where you run the notebook, you will have have the thre files
<pre>
prod_0_MAXIJ1820+070_isgri_spectrum.fits  prod_1_MAXIJ1820+070_isgri_arf.fits  prod_2_MAXIJ1820+070_isgri_rmf.fits
</pre>
that you can analyze with your preferred program.

## Let's extract a lightcurve with large bins

In [ ]:
descr = disp.get_product_description(instrument='isgri',product_name='isgri_lc')

In [ ]:
#We get a light curve with 1000 s time bin and from 50 pointings (note that maximum is 500 !)
light_curve = disp.get_product(instrument="isgri", 
                 product="isgri_lc", 
                 product_type="Real", 
                 osa_version='OSA11.0',
                     RA=275.09142677,
                         DEC=7.18535523,
                    radius = 8,
                     T1=58193.455,
                         T2=58246.892,
                            E1_keV=30,
                            E2_keV=80,
                         T_format= 'mjd',
                     max_pointings=50,
                            time_bin=1000, #time bin in seconds
                     token=token,
                 selected_catalog=api_cat_str)

In [ ]:
#We look at which light curves are produced
light_curve.show()

In [ ]:
#We get the lightcurve that we care about (note that '+' is replaced by 'p' and '-' by 'm')
lc_maxi=light_curve.isgri_lc_0_MAXIJ1820p070

In [ ]:
#We plot the light curve
import numpy as np
plt.figure()
t = lc_maxi.data_unit[1].data['TIME']
dt = lc_maxi.data_unit[1].data['XAX_E']
r = lc_maxi.data_unit[1].data['RATE']
dr = lc_maxi.data_unit[1].data['ERROR']

ind =  (r != 0) & (dr != 0)
t = t[ind]
r = r[ind]
dt = dt[ind]
dr = dr[ind]

title = '%s light curve of %s' %(lc_maxi.data_unit[1].header['DETNAM'], lc_maxi.data_unit[1].header['NAME'])
xlabel = 'Time [IJD = MJD - 51544]'
ylabel = 'Rate [%d-%d keV]' % (lc_maxi.data_unit[1].header['E_MIN'], lc_maxi.data_unit[1].header['E_MAX'])
plt.errorbar(t,r, xerr=dt, yerr=dr, marker='o', color='black', ecolor='black')
plt.xlabel(xlabel)
plt.ylabel(ylabel)
plt.title(title)